<a href="https://colab.research.google.com/github/eistratova/ITMO_STUDY/blob/main/%D0%A4%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%B8_%D0%B4%D0%BB%D1%8F_%D0%A1%D0%BB%D1%83%D1%87%D0%B0%D0%B9%D0%BD%D1%8B%D1%85_%D0%B2%D0%B5%D0%BB%D0%B8%D1%87%D0%B8%D0%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

def mult_rnd_values(P_ksi_eta: np.ndarray , ksi_vals: np.ndarray, eta_vals: np.ndarray) -> np.ndarray:
    """
    Умножение двух Случайных величин кси и эта по совместной таблице распределения
    Функция сортирует значения и складывает вероятности одинаковых значений
    
    Вход: Совместная таблица случайных величин кси и эта
        Значения Кси (axis = 1)
        Значения Эта (axis = 0)
    Выход: [[Значения Кси * Эта],
           [Вероятности значений]]
    """
    mult = np.zeros((2,0))
    for i in range(len(eta_vals)):
        for j in range(len(ksi_vals)):
            # Если вероятность не равна нулю то добавлять новое значение
            if P_ksi_eta[i][j] != 0:
                mult = np.c_[mult, np.array([
                    [eta_vals[i] * ksi_vals[j]],
                    [P_ksi_eta[i][j]]
                ])]
    # Отсортировать массив
    mult = mult[:, np.argsort( mult[0] ) ]
    # Если значения близки, то объединить и сложить вероятности
    mult_short = mult[:,:1] # Первое значение копируем
    for i in range(1, len(mult[0])): # Остальные сверяем с первым
        # Если близки, то суммируем, иначе добавляем новое
        if np.isclose (mult[0][i-1] , mult[0][i] ):
            mult_short[1][-1] += mult[1][i]
        else:
            mult_short = np.c_[mult_short, mult[:,i:i+1] ]
    # Вернуть результат
    return mult_short

def mat_ozidanie( P_ksi: np.ndarray ) -> float:
    """
    Расчитывает математическое ожидание Случайной величины кси
    Вход: двумерный массив [[значения], [вероятности]]
    Выход: Число
    """
    val = 0
    for i in range(len(P_ksi[0])):
        val += P_ksi[0][i] * P_ksi[1][i]
    return val

def dispersion( P_ksi: np.ndarray ) -> float:
    """
    Расчитывает дисперсию Случайной величины кси
    Вход: двумерный массив [[значения], [вероятности]]
    Выход: Число
    """
#   Первый вариант решения не во всех случая выдаёт верный результат
#     P_ksi_kvadr = P_ksi.copy()
#     P_ksi_kvadr[0] = P_ksi_kvadr[0] ** 2
#     return mat_ozidanie(P_ksi_kvadr) - mat_ozidanie(P_ksi) ** 2
    # Второй вариант решения рабочий
    P_ksi_sredn = P_ksi.copy()
    P_ksi_sredn[0] = (P_ksi[0] - mat_ozidanie(P_ksi)) ** 2
    return mat_ozidanie(P_ksi_sredn)

def marginal_raspred(P_ksi_eta: np.ndarray, axis=0) -> np.ndarray:
    """
    Функция выделяет маргинальные веротности по совместной таблице
    распределения случайных величин.
    
    Вход: Совместная таблица случайных величин кси и эта
        axis = 0 или 1 (по рядам или по столбцам)
    Выход: [[Значения Кси или Эта],
           [Вероятности значений]]
    """
    return P_ksi_eta.sum(axis=axis)

def cov(P_ksi_eta: np.ndarray, ksi_vals: np.ndarray, eta_vals: np.ndarray) -> float:
    """
    Функция рассчитывает ковариацию случайных величин Кси и Эта.
    
    Вход: Совместная таблица случайных величин кси и эта
        Значения Кси (axis = 1)
        Значения Эта (axis = 0)
    Выход: значение ковариации
    """
    P_ksi = P_ksi_eta.sum(axis=0)
    P_ksi = np.c_[ksi_vals, P_ksi].T
    P_eta = P_ksi_eta.sum(axis=1)
    P_eta = np.c_[eta_vals, P_eta].T
    mul_ksi_eta = mult_rnd_values (P_ksi_eta, ksi_vals, eta_vals)
    return mat_ozidanie(mul_ksi_eta) - mat_ozidanie(P_ksi) * mat_ozidanie(P_eta)

def koef_correl(P_ksi_eta: np.ndarray, ksi_vals: np.ndarray, eta_vals: np.ndarray) -> float:
    """
    Функция коэффициент корреляции случайных величин Кси и Эта.
    
    Вход: Совместная таблица случайных величин кси и эта
        Значения Кси (axis = 1)
        Значения Эта (axis = 0)
    Выход: значение коэффициента корреляции
    """
    P_ksi = P_ksi_eta.sum(axis=0)
    P_ksi = np.c_[ksi_vals, P_ksi].T
    P_eta = P_ksi_eta.sum(axis=1)
    P_eta = np.c_[eta_vals, P_eta].T
    mul_ksi_eta = mult_rnd_values (P_ksi_eta, ksi_vals, eta_vals)
    cov_ksi_eta = mat_ozidanie(mul_ksi_eta) - mat_ozidanie(P_ksi) * mat_ozidanie(P_eta)
    return cov_ksi_eta / (dispersion(P_ksi)**0.5 * dispersion(P_eta)**0.5)

In [ ]:
# Дано
P_ksi_eta = np.array([
                    [0.1, 0.3,  0.2],
                    [0.1, 0.05, 0],
                    [0,   0.15, 0.1]
                    ])
ksi_vals = np.array([2, 3, 5])
eta_vals = np.array([-1, 1, 4])

In [ ]:
# Маргинальное распределение Кси, по столбцам
P_ksi = np.array([ksi_vals, marginal_raspred(P_ksi_eta, 0)])
P_ksi

array([[2. , 3. , 5. ],
       [0.2, 0.5, 0.3]])

In [ ]:
# Маргинальное распределение Кси, по строкам
P_eta = np.array([ksi_vals, marginal_raspred(P_ksi_eta, 1)])
P_eta

array([[2.  , 3.  , 5.  ],
       [0.6 , 0.15, 0.25]])

In [ ]:
# Распределение P (ksi * eta)
mult_ksi_eta = mult_rnd_values(P_ksi_eta, ksi_vals, eta_vals)
mult_ksi_eta

array([[-5.  , -3.  , -2.  ,  2.  ,  3.  , 12.  , 20.  ],
       [ 0.2 ,  0.3 ,  0.1 ,  0.1 ,  0.05,  0.15,  0.1 ]])

In [ ]:
# Математическое ожидание
mat_ozidanie(mult_ksi_eta)

2.05

In [ ]:
# Дисперсия
dispersion(mult_ksi_eta)

66.3475

In [ ]:
# Ковариация
cov(P_ksi_eta, ksi_vals, eta_vals)

0.17999999999999994

In [ ]:
# Коэффициент корреляции
koef_correl(P_ksi_eta, ksi_vals, eta_vals)

0.07664850422701898